In [33]:
import csv
import os
import pandas as pd
import numpy as np
import rasterio

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# 2016
image_1 = []
label_1 = []
group_1 = []
image_path = '/content/drive/MyDrive/GEE_regen'
label_path = '/content/drive/MyDrive/GEE_Dynamic_2016_three_class'
for filename in os.listdir(image_path):
  image_1.append(os.path.join(image_path,filename))
for filename in os.listdir(label_path):
  label_1.append(os.path.join(label_path,filename))
  group_1.append(0)
df_2016 = pd.DataFrame (list(zip(image_1, label_1, group_1)), columns = ['image_fn', 'label_fn', 'group'])
df_2016.to_csv('data_2016.csv')
!cp data_2016.csv "drive/My Drive/"

In [29]:
# 2019
image_2 = []
label_2 = []
group_2 = []
image_path = '/content/drive/MyDrive/GEE_2019_final'
label_path = '/content/drive/MyDrive/GEE_Dynamic_2019_three_class'
for filename in os.listdir(image_path):
  image_2.append(os.path.join(image_path,filename))
for filename in os.listdir(label_path):
  label_2.append(os.path.join(label_path,filename))
  group_2.append(1)
df_2019 = pd.DataFrame (list(zip(image_2, label_2, group_2)), columns = ['image_fn', 'label_fn', 'group'])
df_2019.to_csv('data_2019.csv')
!cp data_2019.csv "drive/My Drive/"

In [30]:
# both
image_both = []
label_both = []
group_both = []
for value in range(len(image_1)):
  image_both.append(image_1[value])
  image_both.append(image_2[value])
  label_both.append(label_1[value])
  label_both.append(label_2[value])
  group_both.append(group_1[value])
  group_both.append(group_2[value])
df_both = pd.DataFrame (list(zip(image_both, label_both, group_both)), columns = ['image_fn', 'label_fn', 'group'])
df_both.to_csv('data_both.csv')
!cp data_both.csv "drive/My Drive/"

In [ ]:
# convert tif to png
import os
from PIL import Image

yourpath = os.getcwd()
for root, dirs, files in os.walk(yourpath, topdown=False):
    for name in files:
        print(os.path.join(root, name))
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".tiff":
            if os.path.isfile(os.path.splitext(os.path.join(root, name))[0] + ".jpg"):
                print "A jpeg file already exists for %s" % name
            # If a jpeg is *NOT* present, create one from the tiff.
            else:
                outfile = os.path.splitext(os.path.join(root, name))[0] + ".jpg"
                try:
                    im = Image.open(os.path.join(root, name))
                    print "Generating jpeg for %s" % name
                    im.thumbnail(im.size)
                    im.save(outfile, "JPEG", quality=100)
                except Exception, e:
                    print e


In [35]:
sampled_pixels = []

for i, fn in enumerate(image_1):
    if i % 100 == 0:
        print(i, len(image_1))
    with rasterio.open(fn) as f:
        data = f.read().reshape(3,-1)
        
    mask = np.sum(data==0, axis=0) == 3
    data = data[:,~mask]
    num_samples = min(int(1e5), data.shape[1])
    idxs = np.random.choice(data.shape[1], size=num_samples)
    
    pixels = data[:,idxs]
    sampled_pixels.append(pixels)

0 256
100 256
200 256


In [36]:
sampled_pixels = np.concatenate(sampled_pixels, axis=1)

In [37]:
sampled_pixels.mean(axis=1, dtype=np.float64)

array([83.17583934, 79.00489238, 55.64901156])

In [38]:
sampled_pixels.std(axis=1, dtype=np.float64)

array([13.82920992,  9.15737738, 10.08831001])

In [39]:
sampled_pixels = []

for i, fn in enumerate(image_2):
    if i % 100 == 0:
        print(i, len(image_2))
    with rasterio.open(fn) as f:
        data = f.read().reshape(3,-1)
        
    mask = np.sum(data==0, axis=0) == 3
    data = data[:,~mask]
    num_samples = min(int(1e5), data.shape[1])
    idxs = np.random.choice(data.shape[1], size=num_samples)
    
    pixels = data[:,idxs]
    sampled_pixels.append(pixels)

0 256
100 256
200 256


In [40]:
sampled_pixels = np.concatenate(sampled_pixels, axis=1)

In [41]:
sampled_pixels.mean(axis=1, dtype=np.float64)

array([80.80569512, 79.21879715, 54.69975574])

In [42]:
sampled_pixels.std(axis=1, dtype=np.float64)

array([9.3518378 , 5.94109496, 6.79173963])

In [ ]:
img = '/content/00.tif'
with rasterio.open(img) as f:
  data_img = f.read()
data_img = (data_img - sampled_pixels.mean(axis=1, dtype=np.float64))/sampled_pixels.std(axis=1, dtype=np.float64)